In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [5]:
weather_data = pd.read_csv('../data/weather.csv')

col_to_nan = {}
for col in weather_data.columns:
    percent_nan = weather_data[col].isna().sum() / 4017
    
    if percent_nan > .6:
        weather_data.drop(col, axis=1, inplace=True)

to_drop = ['NAME' , 'FMTM', 'FMTM_ATTRIBUTES', 'AWND', 'LONGITUDE', 'LATITUDE', 'ELEVATION', 'AWND_Attributes', 
           'SNOW_ATTRIBUTES', 'SNWD', 'PRCP_ATTRIBUTES', 'SNOW_ATTRIBUTES', 'TAVG_ATTRIBUTES', 
           "WDF2","WDF2_ATTRIBUTES","WDF5","WDF5_ATTRIBUTES", 'TAVG',
           "WESD_ATTRIBUTES", 'WESD', "WSF2","WSF2_ATTRIBUTES","WSF5","WSF5_ATTRIBUTES", 'STATION']


for col in weather_data.columns:
    if col in to_drop or 'ATTRIBUTES' in col:
        weather_data.drop(col, axis=1, inplace=True)

        
weather_data['DATE'] = weather_data['DATE'].apply(lambda x: f'{x[5:7]}/{x[8:]}/{x[:4]}')
weather_data.head(10)

,DATE,PRCP,TMAX,TMIN
0,01/01/2009,0.0,54,45
1,01/02/2009,0.0,53,48
2,01/03/2009,0.0,60,50
3,01/04/2009,0.0,63,46
4,01/05/2009,0.0,58,42
5,01/06/2009,0.0,59,45
6,01/07/2009,0.0,63,46
7,01/08/2009,0.0,57,47
8,01/09/2009,0.0,64,50
9,01/10/2009,0.0,71,44


In [5]:
flight_data = pd.read_csv('../data/Detailed_Statistics_Arrivals.csv', skiprows=7)
flight_data.drop(441999, inplace=True)
flight_data

flight_data.set_axis(['Carrier Code', 'DATE', 'Flight Number', 'Tail Number',
       'Origin Airport', 'Scheduled Arrival Time', 'Actual Arrival Time',
       'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)',
       'Arrival Delay (Minutes)', 'Wheels-on Time', 'Taxi-In time (Minutes)',
       'Delay Carrier (Minutes)', 'Delay Weather (Minutes)',
       'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)',
       'Delay Late Aircraft Arrival (Minutes)'], axis=1, inplace=True)

# all_data = flight_data.merge(weather_data,on='DATE',how='left')
all_data = flight_data.set_index('DATE').join(weather_data.set_index('DATE'))
all_data['TAVG'] = (all_data['TMAX'] + all_data['TMIN']) / 2

all_data = all_data.drop(['TMAX', 'TMIN'], axis=1)
all_data

,Carrier Code,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Wheels-on Time,Taxi-In time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),PRCP,TAVG
DATE,,,,,,,,,,,,,,,,,,
01/01/2009,WN,12.0,N414WN,ABQ,12:45,13:00,125.0,112.0,15.0,12:54,6.0,1.0,0.0,0.0,0.0,14.0,0.0,49.5
01/01/2009,WN,21.0,N401WN,HOU,14:00,13:56,215.0,200.0,-4.0,13:50,6.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
01/01/2009,WN,72.0,N399WN,OAK,11:20,11:14,80.0,68.0,-6.0,11:08,6.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
01/01/2009,WN,75.0,N318SW,SMF,19:15,19:11,85.0,76.0,-4.0,19:03,8.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
01/01/2009,WN,87.0,N430WN,MDW,14:05,13:53,275.0,255.0,-12.0,13:46,7.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/2019,WN,6467.0,N734SA,OAK,07:30,07:15,90.0,76.0,-15.0,07:11,4.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
12/31/2019,WN,6516.0,N267WN,DEN,18:55,18:39,155.0,125.0,-16.0,18:31,8.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
12/31/2019,WN,6614.0,N8650F,MSY,20:25,20:17,265.0,251.0,-8.0,20:09,8.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0


In [6]:
len(set(all_data.index))

4017